In [54]:
#TODO: understand timing convetions and problems with bonds
# remove plainvanillabond and add a stub to fixedrate and floatrate notes
# add multiple possible functions to discount like linear discounting.

In [2]:
import QuantLib as ql

In [12]:
effectiveDate = ql.Date(15,6,2020)
terminationDate = ql.Date(30,6,2022)
frequency = ql.Period('6M')
calendar = ql.TARGET()
convention = ql.Unadjusted
terminationDateConvention = ql.Unadjusted
rule = ql.DateGeneration.Forward
endOfMonth = False
schedule = ql.Schedule(effectiveDate, terminationDate, frequency, calendar, convention, terminationDateConvention, rule, endOfMonth)

In [13]:
schedule.dates()

(Date(15,6,2020),
 Date(15,12,2020),
 Date(15,6,2021),
 Date(15,12,2021),
 Date(15,6,2022),
 Date(30,6,2022))

In [31]:
start_date = ql.Date(1,7,2024)
end_date = ql.Date(21,7,2024)
tenor = ql.Period(3, ql.Days)
calendar = ql.NullCalendar()
convention = ql.Unadjusted
termination_convention = ql.Unadjusted
rule = ql.DateGeneration.Forward
end_of_month = False

# Define stub dates
first_stub_date = ql.Date(1, 7, 2024)
next_to_last_stub_date = ql.Date(16, 7, 2024)

# Create the schedule with stub dates
schedule = ql.Schedule(start_date, end_date, tenor, calendar, convention,
                       termination_convention, rule, end_of_month, first_stub_date, next_to_last_stub_date)

In [32]:
schedule.dates()

(Date(1,7,2024),
 Date(4,7,2024),
 Date(7,7,2024),
 Date(10,7,2024),
 Date(13,7,2024),
 Date(16,7,2024),
 Date(21,7,2024))

Basic Bond Example

In [35]:
# basic example of a bond
start_date = ql.Date(1,1,2024)
end_date = ql.Date(1,1,2025)
tenor = ql.Period('6M')
calendar = ql.NullCalendar()
convention = ql.Unadjusted
termination_convention = ql.Unadjusted
rule = ql.DateGeneration.Forward
end_of_month = False


# Create the schedule with stub dates
schedule = ql.Schedule(start_date, end_date, tenor, calendar, convention,
                       termination_convention, rule, end_of_month)
schedule.dates()

(Date(1,1,2024), Date(1,7,2024), Date(1,1,2025))

In [42]:
interest = ql.FixedRateLeg(schedule, ql.Actual365Fixed(), [100.], [0.05])

In [43]:
bond = ql.Bond(0, ql.NullCalendar(), start_date, interest)

In [44]:
for cf in bond.cashflows():
    print(f"Date: {cf.date()}, Amount: {cf.amount()}")

Date: July 1st, 2024, Amount: 2.4931506849315
Date: January 1st, 2025, Amount: 2.520547945205487
Date: January 1st, 2025, Amount: 100.0


In [67]:
interest_rate = 0.03  # 3% flat rate
discount_curve = ql.FlatForward(ql.Date(1,1,2024), ql.QuoteHandle(ql.SimpleQuote(interest_rate)), ql.Actual365Fixed())


In [68]:
bond_engine = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(discount_curve))

# Assign the pricing engine to the bond
bond.setPricingEngine(bond_engine)

In [69]:
bond.dirtyPrice()

101.48098747548846

In [71]:
date = ql.Date(1, 1, 2024)
discount_factor = discount_curve.discount(date)
discount_factor

1.0

In [49]:
(ql.Date(1,7,2024) - ql.Date(1,1,2024))/365*5

2.493150684931507

In [26]:
from datetime import date, datetime, timedelta
from rivapy.tools import Schedule, Period
from rivapy.tools.enums import RollConvention
from rivapy.tools.datetools import DayCounter
from rivapy.tools.enums import DayCounterType
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
DayCounterType.Thirty360ISDA.value

'30360ISDA'

In [34]:
dc = DayCounter('Act252')

NotImplementedError: Act252 not yet implemented.

In [32]:
dc._yf

<function rivapy.tools.datetools.DayCounter.yf_30360ISDA(d1: Union[datetime.date, datetime.datetime], d2: Union[datetime.date, datetime.datetime]) -> float>

In order to understand RIVAPY's `Schedule` class, we want to create dates seperated by a specific time period. We define the time period between two dates to be 3 days. This can be done using the `Period` class and passing `Period(years=0, months=0, days=3)` as a parameter for the `time_period` argument of the `Schedule` class. The `start_day` is then set to be 01.07.2024 while the `end_day` is 21.07.2024. With this setup not every period has the same length. For simplicity we do not modify the days, if a period falls on a holiday or weekend. Hence, we set `business_day_convention=RollConvention.UNADJUSTED`.

We first set the parameters `backwards` and `stub` to `False`.

In [52]:
schedule = Schedule(start_day=date(2024, 7, 1), end_day=date(2024, 7, 21), time_period=Period(0, 0, 3), backwards=False, stub=False, business_day_convention=RollConvention.UNADJUSTED)
schedule.generate_dates(False)

[datetime.datetime(2024, 7, 1, 0, 0),
 datetime.datetime(2024, 7, 4, 0, 0),
 datetime.datetime(2024, 7, 7, 0, 0),
 datetime.datetime(2024, 7, 10, 0, 0),
 datetime.datetime(2024, 7, 13, 0, 0),
 datetime.datetime(2024, 7, 16, 0, 0),
 datetime.datetime(2024, 7, 21, 0, 0)]

We see that the first period each period except the last contains has the wanted period length of 3 days. With the parameter `stub=False` the last period is not allowed to contain less than 3 days.

If we change the parameter `stub` to `True`, we see that the last period contains now less than 3 days.

In [49]:
schedule = Schedule(start_day=date(2024, 7, 1), end_day=date(2024, 7, 21), time_period=Period(0, 0, 3), backwards=False, stub=True, business_day_convention=RollConvention.UNADJUSTED)
schedule.generate_dates(False)

[datetime.datetime(2024, 7, 1, 0, 0),
 datetime.datetime(2024, 7, 4, 0, 0),
 datetime.datetime(2024, 7, 7, 0, 0),
 datetime.datetime(2024, 7, 10, 0, 0),
 datetime.datetime(2024, 7, 13, 0, 0),
 datetime.datetime(2024, 7, 16, 0, 0),
 datetime.datetime(2024, 7, 19, 0, 0),
 datetime.datetime(2024, 7, 21, 0, 0)]

Setting `stub` back to `False` and changing the parameter `backwards` to `True`, reverses the behaviour. Instead of the last period, the first period now has contains more than 3 days, while the last period contains exatcly 3 days.

In [53]:
schedule = Schedule(start_day=date(2024, 7, 1), end_day=date(2024, 7, 21), time_period=Period(0, 0, 3), backwards=True, stub=False, business_day_convention=RollConvention.UNADJUSTED)
schedule.generate_dates(False)

[datetime.datetime(2024, 7, 1, 0, 0),
 datetime.datetime(2024, 7, 6, 0, 0),
 datetime.datetime(2024, 7, 9, 0, 0),
 datetime.datetime(2024, 7, 12, 0, 0),
 datetime.datetime(2024, 7, 15, 0, 0),
 datetime.datetime(2024, 7, 18, 0, 0),
 datetime.datetime(2024, 7, 21, 0, 0)]

In [50]:
schedule = Schedule(start_day=date(2024, 7, 1), end_day=date(2024, 7, 21), time_period=Period(0, 0, 3), backwards=True, stub=True, business_day_convention=RollConvention.UNADJUSTED)
schedule.generate_dates(False)

[datetime.datetime(2024, 7, 1, 0, 0),
 datetime.datetime(2024, 7, 3, 0, 0),
 datetime.datetime(2024, 7, 6, 0, 0),
 datetime.datetime(2024, 7, 9, 0, 0),
 datetime.datetime(2024, 7, 12, 0, 0),
 datetime.datetime(2024, 7, 15, 0, 0),
 datetime.datetime(2024, 7, 18, 0, 0),
 datetime.datetime(2024, 7, 21, 0, 0)]

In [5]:
d1 = datetime(2024, 1, 1)
d2 = datetime(2024, 1, 20)

In [6]:
(d2 - d1).days

19

In [8]:
date(1900, 1, 1) + timedelta(days=35728)

datetime.date(1997, 10, 27)